In [20]:
import psycopg2

# Verbindung zur PostgreSQL-Datenbank herstellen
connection = None  # Variable vorinitialisieren

try:
    connection = psycopg2.connect(
        host="130.61.234.133",      # IP-Adresse der Datenbank
        port="5432",                # Portnummer
        database="crawler_db",      # Name der Datenbank
        user="crawler_user",          # Benutzername
        password="crawler_password" # Passwort
    )
    print("Verbindung zur Datenbank erfolgreich hergestellt.")
    
    # Cursor-Objekt erstellen, um SQL-Befehle auszuführen
    cursor = connection.cursor()

    ## Beerbrands
    # SQL-Abfrage ausführen, um Daten abzurufen
    cursor.execute("SELECT * FROM beerbrands")

    # Alle Zeilen der Abfrageergebnisse abrufen
    data_beerbrands = cursor.fetchall()

    ## Beers
    cursor.execute("SELECT * FROM beers")
    data_beers = cursor.fetchall()

    ## Formats
    cursor.execute("SELECT * FROM formats")
    data_formats = cursor.fetchall()

    ## Prices
    cursor.execute("SELECT * FROM prices")
    data_prices = cursor.fetchall()

    ## Resellers
    cursor.execute("SELECT * FROM resellers")
    data_resellers = cursor.fetchall()

    # Daten ausgeben
    #for zeile in daten:
        #print(zeile)
    
except Exception as error:
    print(f"Fehler beim Verbinden zur Datenbank: {error}")
    
finally:
    # Verbindung schließen, falls sie geöffnet wurde
    if connection is not None:
        cursor.close()
        connection.close()
        print("Verbindung zur Datenbank geschlossen.")



Verbindung zur Datenbank erfolgreich hergestellt.
Verbindung zur Datenbank geschlossen.


In [21]:
import pandas as pd

data_beerbrands = pd.DataFrame(data_beerbrands, columns=['brand_id', 'brand', 'id'])
display(data_beerbrands)

## Beers
# Umwandlung in ein DataFrame
data_beers = pd.DataFrame(data_beers, columns=['beer_id', 'name', 'alcohol_percentange'])
display(data_beers)

## Formats
data_formats = pd.DataFrame(data_formats, columns=['format_id', 'beer_id', 'quantity', 'unit'])
display(data_formats)

## Prices
data_prices = pd.DataFrame(data_prices, columns=['prices_id', 'beer_id', 'format_id', 'reseller_id', 'price', 'currency', 'date', 'url'])
display(data_prices)

## Resellers
data_resellers = pd.DataFrame(data_resellers, columns=['reseller_id', 'reseller_name', 'zipcode'])
display(data_resellers)

,brand_id,brand,id
0,1,Krombacher,1
1,3,Aktien,2
2,4,Allgäuer,3
3,5,Alpirsbacher,4
4,6,Altenburger,5
...,...,...,...
305,279,Eschenbach,304
306,280,Flötzinger,305
307,281,Wildbräu,306
308,282,Schweiger,307


,beer_id,name,alcohol_percentange
0,1,Paulaner Hefe-Weißbier Naturtrüb Glas,0.00
1,2,Ustersbacher Bayerisch Hefeweizen Glas,5.50
2,3,"Gösser Naturradler, Dose Einweg",2.00
3,4,Benediktiner Hell Glas,5.00
4,5,"Zirndorfer Landbier 4,9 % vol. Glas",0.00
...,...,...,...
1469,1628,Feuerwehr Bier,None
1470,1629,Weihnachts-Bier,None
1471,1630,DDR Bier im witzigen Ostalgie,None
1472,1631,Weihnachts-Pils Festretter Weihnachtsbier im B...,None


,format_id,beer_id,quantity,unit
0,1,1,10.00,l
1,2,2,10000.00,ml
2,3,3,500.00,ml
3,4,4,10.00,l
4,5,5,10.00,l
...,...,...,...,...
2598,2599,1628,1.00,L
2599,2600,1629,1.00,L
2600,2601,1630,1.00,L
2601,2602,1631,1.00,L


,prices_id,beer_id,format_id,reseller_id,price,currency,date,url
0,1,1,1,1,18.43,€,2024-09-17,https://produkte.metro.de/shop/pv/BTY-X388641/...
1,2,2,2,1,15.93,€,2024-09-17,https://produkte.metro.de/shop/pv/BTY-X574359/...
2,3,3,3,1,1.13,€,2024-09-17,https://produkte.metro.de/shop/pv/BTY-X320745/...
3,4,4,4,1,20.15,€,2024-09-17,https://produkte.metro.de/shop/pv/BTY-X322512/...
4,5,5,5,1,16.31,€,2024-09-17,https://produkte.metro.de/shop/pv/BTY-X492469/...
...,...,...,...,...,...,...,...,...
53072,53073,350,403,153,20.60,€,2024-09-25,https://www.biertaxi-duesseldorf.de/shop/artic...
53073,53074,655,1252,153,13.60,€,2024-09-25,https://www.biertaxi-duesseldorf.de/shop/artic...
53074,53075,712,1248,153,19.35,€,2024-09-25,https://www.biertaxi-duesseldorf.de/shop/artic...
53075,53076,445,1250,153,27.95,€,2024-09-25,https://www.biertaxi-duesseldorf.de/shop/artic...


,reseller_id,reseller_name,zipcode
0,1,Metro,86156
1,2,Metro,13599
2,3,Metro,10243
3,4,Metro,12277
4,5,Metro,38122
...,...,...,...
148,141,Biertaxi,online
149,150,Edeka24,
150,151,Kaufland,
151,152,Netto Online,
